# import libraries

In [1]:
import numpy as np
import pandas as pd
import random
from pathlib import Path
import json
from collections import OrderedDict, defaultdict
import csv
import os
from tqdm import tqdm
from scipy.stats import pearsonr, spearmanr
import glob
import json
from matplotlib import pyplot as plt
import seaborn as sns
from matplotlib.patches import Rectangle

# read test data and corresponding expressions

In [2]:
filename = 'filtered_test_data_with_MAUDE_expression.txt'
with open(filename) as f:
    reader = csv.reader(f, delimiter="\t")
    lines = list(reader)

filtered_tagged_sequences = [line[0] for line in lines]
expressions = [line[1] for line in lines]

GROUND_TRUTH_EXP = np.array([float(expressions[i]) for i in range(len(filtered_tagged_sequences))])
print(len(GROUND_TRUTH_EXP))

print(max(GROUND_TRUTH_EXP))
print(min(GROUND_TRUTH_EXP))

71103
1.65983547290171
-1.40175772469441


# read different promoter classes ids

In [3]:
df = pd.read_csv('test_subset_ids/high_exp_seqs.csv')
high = list(df['pos'])
high = np.unique(np.array(high))

df = pd.read_csv('test_subset_ids/low_exp_seqs.csv')
low = list(df['pos'])
low = np.unique(np.array(low))

df = pd.read_csv('test_subset_ids/yeast_seqs.csv')
yeast = list(df['pos'])
yeast = np.unique(np.array(yeast))

df = pd.read_csv('test_subset_ids/all_random_seqs.csv')
random = list(df['pos'])
random = np.unique(np.array(random))

df = pd.read_csv('test_subset_ids/challenging_seqs.csv')
challenging = list(df['pos'])
challenging = np.unique(np.array(challenging))

df = pd.read_csv('test_subset_ids/all_SNVs_seqs.csv')
SNVs_alt = list(df['alt_pos'])
SNVs_ref = list(df['ref_pos'])
SNVs = list(set(list(zip(SNVs_alt, SNVs_ref))))

df = pd.read_csv('test_subset_ids/motif_perturbation.csv')
motif_perturbation_alt = list(df['alt_pos'])
motif_perturbation_ref = list(df['ref_pos'])
motif_perturbation = list(set(list(zip(motif_perturbation_alt, motif_perturbation_ref))))

df = pd.read_csv('test_subset_ids/motif_tiling_seqs.csv')
motif_tiling_alt = list(df['alt_pos'])
motif_tiling_ref = list(df['ref_pos'])
motif_tiling = list(set(list(zip(motif_tiling_alt, motif_tiling_ref))))

In [4]:
print(len(high))
print(len(low))
print(len(yeast))
print(len(random))
print(len(challenging))
print(len(motif_perturbation))
print(len(motif_tiling))
print(len(SNVs))

968
997
578
6349
1953
3287
2624
44340


# Get the different promoter class ids used in the public leaderboard of the DREAM challenge

In [5]:
with open('public_leaderboard_ids/high_exp_indices.json', 'r') as f:
    public_high = [int(indice) for indice in list(json.load(f).keys())]

with open('public_leaderboard_ids/low_exp_indices.json', 'r') as f:
    public_low = [int(indice) for indice in list(json.load(f).keys())]

with open('public_leaderboard_ids/yeast_exp_indices.json', 'r') as f:
    public_yeast = [int(indice) for indice in list(json.load(f).keys())]

with open('public_leaderboard_ids/random_exp_indices.json', 'r') as f:
    public_random = [int(indice) for indice in list(json.load(f).keys())]

with open('public_leaderboard_ids/challenging_exp_indices.json', 'r') as f:
    public_challenging = [int(indice) for indice in list(json.load(f).keys())]
    
with open('public_leaderboard_ids/SNVs_exp_indices.json', 'r') as f:
    public_SNVs = [(int(indice.split(',')[0]), int(indice.split(',')[1])) for indice in list(json.load(f).keys())]

with open('public_leaderboard_ids/motif_perturbation_exp_indices.json', 'r') as f:
    public_motif_perturbation = [(int(indice.split(',')[0]), int(indice.split(',')[1])) for indice in list(json.load(f).keys())]

with open('public_leaderboard_ids/motif_tiling_exp_indices.json', 'r') as f:
    public_motif_tiling = [(int(indice.split(',')[0]), int(indice.split(',')[1])) for indice in list(json.load(f).keys())]

In [6]:
print(len(public_high))
print(len(public_low))
print(len(public_yeast))
print(len(public_random))
print(len(public_challenging))
print(len(public_SNVs))
print(len(public_motif_perturbation))
print(len(public_motif_tiling))

96
99
208
733
195
4670
328
263


# Get the final test dataset used for private evaluation in the DREAM Chalenge

In [7]:
public_single_indices = public_high + public_low + public_yeast + public_random + public_challenging
public_double_indices = public_SNVs + public_motif_perturbation + public_motif_tiling

public_indices = []

for indice in public_double_indices:
    public_indices.append(indice[0])
    public_indices.append(indice[1])

for indice in public_single_indices:
    public_indices.append(indice)

public_indices = list(set(public_indices))

final_high = [exp for exp in high if exp not in public_indices]
final_low = [exp for exp in low if exp not in public_indices]
final_yeast = [exp for exp in yeast if exp not in public_indices]
final_random = [exp for exp in random if exp not in public_indices]
final_challenging = [exp for exp in challenging if exp not in public_indices]
final_SNVs = [exp for exp in SNVs if exp not in public_double_indices]
final_motif_perturbation = [exp for exp in motif_perturbation if exp not in public_double_indices]
final_motif_tiling = [exp for exp in motif_tiling if exp not in public_double_indices]
final_all = [exp for exp in list(range(len(GROUND_TRUTH_EXP))) if exp not in public_indices]

In [8]:
print(len(final_high))
print(len(final_low))
print(len(final_yeast))
print(len(final_random))
print(len(final_challenging))
print(len(final_SNVs))
print(len(final_motif_perturbation))
print(len(final_motif_tiling))
print(len(final_all))

872
898
370
5616
1758
39904
2959
2361
62058


In [9]:
len(set(final_SNVs))

39904

# Performance on final test dataset

In [10]:
def calculate_correlations(index_list, expressions, GROUND_TRUTH_EXP):
    PRED_DATA = OrderedDict()
    GROUND_TRUTH = OrderedDict()

    for j in index_list:
        PRED_DATA[str(j)] = float(expressions[j])
        GROUND_TRUTH[str(j)] = float(GROUND_TRUTH_EXP[j])

    pearson = pearsonr(list(GROUND_TRUTH.values()), list(PRED_DATA.values()))[0]
    spearman = spearmanr(list(GROUND_TRUTH.values()), list(PRED_DATA.values()))[0]

    return pearson, spearman


def calculate_diff_correlations(pair_list, expressions, GROUND_TRUTH_EXP):
    Y_pred_selected = []
    expressions_selected = []

    for pair in pair_list:
        ref, alt = pair[0], pair[1]
        Y_pred_selected.append(expressions[alt] - expressions[ref])
        expressions_selected.append(GROUND_TRUTH_EXP[alt] - GROUND_TRUTH_EXP[ref])

    Y_pred_selected = np.array(Y_pred_selected)
    expressions_selected = np.array(expressions_selected)

    pearson = pearsonr(expressions_selected, Y_pred_selected)[0]
    spearman = spearmanr(expressions_selected, Y_pred_selected)[0]

    return pearson, spearman

In [11]:
# Read expressions
sequences = []
expressions = []
filename = 'reference model predictions.txt'

with open(filename) as f:
    lines = f.readlines()

for j in range(len(lines)):
    exp = lines[j].split('\t')[1].split('\n')[0]
    expressions.append(float(exp))

expressions = np.array(expressions)

# Calculate correlations
pearson, spearman = calculate_correlations(final_all, expressions, GROUND_TRUTH_EXP)
high_pearson, high_spearman = calculate_correlations(final_high, expressions, GROUND_TRUTH_EXP)
low_pearson, low_spearman = calculate_correlations(final_low, expressions, GROUND_TRUTH_EXP)
yeast_pearson, yeast_spearman = calculate_correlations(final_yeast, expressions, GROUND_TRUTH_EXP)
random_pearson, random_spearman = calculate_correlations(final_random, expressions, GROUND_TRUTH_EXP)
challenging_pearson, challenging_spearman = calculate_correlations(final_challenging, expressions, GROUND_TRUTH_EXP)

# Calculate difference correlations
SNVs_pearson, SNVs_spearman = calculate_diff_correlations(final_SNVs, expressions, GROUND_TRUTH_EXP)
motif_perturbation_pearson, motif_perturbation_spearman = calculate_diff_correlations(final_motif_perturbation, expressions, GROUND_TRUTH_EXP)
motif_tiling_pearson, motif_tiling_spearman = calculate_diff_correlations(final_motif_tiling, expressions, GROUND_TRUTH_EXP)


# Calculate scores
pearsons_score = (pearson**2 + 0.3 * high_pearson**2 + 0.3 * low_pearson**2 + 0.3 * yeast_pearson**2 + 
                  0.3 * random_pearson**2 + 0.5 * challenging_pearson**2 + 1.25 * SNVs_pearson**2 + 
                  0.3 * motif_perturbation_pearson**2 + 0.4 * motif_tiling_pearson**2) / 4.65


spearmans_score = (spearman + 0.3 * high_spearman + 0.3 * low_spearman + 0.3 * yeast_spearman 
                   + 0.3 * random_spearman + 0.5 * challenging_spearman + 1.25 * SNVs_spearman
                   + 0.3 * motif_perturbation_spearman + 0.4 * motif_tiling_spearman) / 4.65

# Print scores
print('******************************************************')
print('Pearson Score: {}\n'.format(pearsons_score))
print('Spearman Score: {}\n'.format(spearmans_score))
print('******************************************************')
print('all r: {}\n'.format(pearson))
print('all r\u00b2: {}\n'.format(pearson**2))
print('all \u03C1: {}\n'.format(spearman))
print('******************************************************')
print('high r: {}\n'.format(high_pearson))
print('low r: {}\n'.format(low_pearson))
print('yeast r: {}\n'.format(yeast_pearson))
print('random r: {}\n'.format(random_pearson))
print('challenging r: {}\n'.format(challenging_pearson))
print('SNVs r: {}\n'.format(SNVs_pearson))
print('motif perturbation r: {}\n'.format(motif_perturbation_pearson))
print('motif tiling r: {}\n'.format(motif_tiling_pearson))
print('******************************************************')
print('high \u03C1: {}\n'.format(high_spearman))
print('low \u03C1: {}\n'.format(low_spearman))
print('yeast \u03C1: {}\n'.format(yeast_spearman))
print('random \u03C1: {}\n'.format(random_spearman))
print('challenging \u03C1: {}\n'.format(challenging_spearman))
print('SNVs \u03C1: {}\n'.format(SNVs_spearman))
print('motif perturbation \u03C1: {}\n'.format(motif_perturbation_spearman))
print('motif tiling \u03C1: {}\n'.format(motif_tiling_spearman))
print('******************************************************')

******************************************************
Pearson Score: 0.6952878620679598

Spearman Score: 0.7623009637180792

******************************************************
all r: 0.9374894920646527

all r²: 0.8788865477316405

all ρ: 0.9383488876303813

******************************************************
high r: 0.5139707890339921

low r: 0.5155473085120057

yeast r: 0.8001240028362957

random r: 0.954834186712707

challenging r: 0.8776323764285696

SNVs r: 0.7883007968891876

motif perturbation r: 0.9464865178914659

motif tiling r: 0.8645815350127515

******************************************************
high ρ: 0.3050549170685601

low ρ: 0.42155030304354485

yeast ρ: 0.7968595982198527

random ρ: 0.9557912874062111

challenging ρ: 0.8572264460906948

SNVs ρ: 0.6444852159552136

motif perturbation ρ: 0.9300690712696714

motif tiling ρ: 0.8733332439174283

******************************************************
